# h2o benchmarks - query 3

In [1]:
import coiled
import dask
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(
    name="powers-crt-004",
    software="crt-004",
    n_workers=16,
    worker_vm_types=["i3.large"],
)

Output()

In [3]:
client = dask.distributed.Client(cluster)

In [4]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/h2o-benchmark/N_1e9_K_1e2_parquet",
    storage_options={"anon": True, "use_ssl": True},
    columns=["id3", "v1", "v3"],
    engine="pyarrow",
)

In [5]:
ddf.memory_usage(deep=True).compute().sum()

223957731090

In [6]:
len(ddf)

1000000000

In [8]:
ddf.dtypes

id3    category
v1        Int32
v3      float64
dtype: object

In [9]:
ddf[["id3"]].describe().compute()

KeyboardInterrupt: 

In [10]:
%%time
ddf.groupby("id3", dropna=False, observed=True).agg(
    {"v1": "sum", "v3": "mean"}
).compute()

KilledWorker: ("('aggregate-combine-c8c78083a7604c33195ca98e7a7a8b6b', 2, 3)", <WorkerState 'tls://10.4.8.147:44373', name: powers-crt-004-worker-0f8b7e3d6d, status: closed, memory: 0, processing: 3>)

In [ ]:
%%time
ddf[["id3"]].nunique().compute()

## Try with pyarrow[string] column

In [11]:
ddf["id3"] = ddf["id3"].astype("string[pyarrow]")

In [12]:
ddf.memory_usage(deep=True).compute().sum()

37000000000

In [13]:
ddf.dtypes

id3     string
v1       Int32
v3     float64
dtype: object

In [14]:
%%time
ddf.repartition(200).groupby("id3", dropna=False, observed=True).agg(
    {"v1": "sum", "v3": "mean"}
).compute()

KilledWorker: ("('aggregate-combine-bda23b3b921d6f957f004356c9498fae', 0, 2)", <WorkerState 'tls://10.4.5.130:38813', name: powers-crt-004-worker-e90d0c3b58, status: closed, memory: 0, processing: 1>)